In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from bs4 import BeautifulSoup
from urllib.request import Request, urlopen
import datetime
import string
from string import digits
import collections
import scipy.stats as scs
import cc_pipeline as P
import time
import random
import pickle
from pprint import pprint
from collections import Counter

#sentiment and language
import nltk
from nltk import word_tokenize
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
import spacy
import vaderSentiment
from langdetect import detect
from gensim.models import Word2Vec
from gensim import corpora
from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer
from spacy import displacy

from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer, TfidfVectorizer
from sklearn.decomposition import PCA, KernelPCA
from sklearn.pipeline import Pipeline
from scipy.spatial.distance import cdist
from sklearn.cluster import KMeans
from sklearn.decomposition import TruncatedSVD
from sklearn.decomposition import NMF, LatentDirichletAllocation
from gensim.models.ldamodel import LdaModel
from sklearn.model_selection import GridSearchCV
from sklearn.feature_selection import chi2
import knee_locator

#plotting
from bokeh.plotting import figure, show, output_file, output_notebook, ColumnDataSource
from bokeh.models import HoverTool, BoxSelectTool
from wordcloud import WordCloud, STOPWORDS, ImageColorGenerator
from PIL import Image
import pyLDAvis.sklearn
import pyLDAvis.gensim as gensimvis
import pyLDAvis
import umap

nltk.download('punkt')

[nltk_data] Downloading package punkt to /home/ubuntu/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /home/ubuntu/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to /home/ubuntu/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [30]:
def inline_text(show_raw):
    
    '''returns show text without timestamps'''
    
    temp = " ".join( ["\n".join( x.split("\n")[2:] ) for x in show_raw.split("\n\n")] )
    temp = temp.split('\n')
    temp = " ".join(temp)
    return temp

def clean_for_spacy(text_list):
    
    '''cleans all text and creates new column in dataframe'''
    
    doc_list = []
    for word in text_list:
        doc_list.append(inline_text(word))
    return doc_list

def clean_for_spacy_lower(text_list):
    
    '''cleans all text and creates new column in dataframe'''
    
    doc_list = []
    for word in text_list:
        doc_list.append(inline_text(word).lower())
    return doc_list

def lang_detect(doc_series):
    
    lang = []
    for x in doc_series:
        eng = 'en'
        span = 'es'

        try:
            if detect(x) == eng:
                lang.append(eng)
            else:
                lang.append(span)
        except:
            lang.append(None)
            
    return lang

def get_orgs(chunks):
    
    nlp = spacy.load('en_core_web_sm')
    orgs = []
    
    for chunk in chunks:
        document = nlp(chunk)
        labels = set([w.label_ for w in document.ents]) 

        for label in labels: 

            temp_entities = [e for e in document.ents if label==e.label_] 
            temp_entities = list(set(temp_entities)) 

            if label == 'ORG':
                orgs.append(str(temp_entities))
                
    orgs = " ".join(orgs)
    return orgs

In [44]:
sent_df = pd.read_csv('data/cc_1000_text.csv', encoding='utf=8')

In [45]:
#english only (for testing)

doc_series = pd.Series(sent_df['text'].values)
language = lang_detect(doc_series)
sent_df['language'] = language
english = sent_df[sent_df['language'] == 'en']

In [46]:
temp_text = english['text'].values
sent_text = clean_for_spacy(temp_text)
sent_text_lower = clean_for_spacy_lower(temp_text)

In [47]:
corpus = " ".join(sent_text)

In [41]:
#break into chunks for spaCy

n = 100000
chunks = [corpus[i:i+n] for i in range(0, len(corpus), n)]
len(chunks)

12

In [42]:
org_list = get_orgs(chunks)
org_list

'[\xa0, I-84 AT, PUBLIC, NEW \xa0, VIDEO, HOT, ESPERANZA, ARMSTRONG, A GRAIN OF SALT, HAPPEN, SHY OF, \xa0, Quick six Wide, Lincoln, Deb, REMAIN, \xa0, Otezla, SIGHS, OUT, SEE, BEST, GIANT, SIGHS, WORLD, CDC, ABC NEWS, BRAIN, BETTER CONSISTENCY, Deb: MY KIDS WOULD, PEPPARKAKOR, \xa0, KILLING 58 PEOPLE, BARRE, TIDE, MEDAL, Lincoln, MAYA, LL, DAD, KNOW, SEC, BALL, \xa0, THE TIME OF YEAR, \xa0, \xa0, \xa0, COMPANY, CHALLENGES, BUTT, HIGHEST, \xa0, \xa0, INCREDIBLE, AMERICAN HERO’S, LUMP OF COAL, AMERICAN HERO’S, SELF, DO, LOT OF, \xa0, BIG ON, SEE, REMAIN, Otezla, KNOW, SET, ACTIVE, TAP, OPENED FIRE, \xa0, AETNA, LOOSE, DEFENSE, CHALLENGE, MERRY CHRISTMAS, THE BATTLE \xa0, \xa0, CYCLE OF VIOLENCE, BEEPER BEEPS, APOLLO 11, CASA, FOCUS, KNOW, WHOLE FOODS, EARTH, \xa0SECRET SERVICE, Thompson, FATALITY, NBC, @LINCOLNGRAVES, NATALIE, WE, UP, THE SAME \xa0IN A LOT OF WAYS, HOT TODAY, OUT, HIM, Lincoln, GOT, SEE, EVERYONE NOW STAFF NRYONE, LL, CAMERA, Decision Guide, \xa0, YOURS OR MINE, KNOW, N